In [56]:
import pandas as pd
import numpy as np

In [57]:
## Stock Split on GOOGL UW (Alphabet Inc. -A-). The 20 for 1 stock split was adjusted.
2461.48 / 20, 5000 / (2461.48 / 20)

(123.074, 40.62596486666558)

## Basic descriptions

In [58]:
denomination = 5000
barrier = 0.59
# being the date on which the Strike and the Barrier and the Ratio is fixed,
# and from which date the Complex Products may be traded
initial_fixing_date = "2022-04-25" 
final_fixing_date = "2023-07-26"
final_redemption_date = "2023-08-02"

issue_date = "2022-05-02"

coupon_rate = 0.11 / 4

coupon_payment_dates = [
    "2022-08-02",
    "2022-11-02",
    "2023-02-02",
    "2023-05-02",
    "2023-08-02"
]

early_redemption_dates = [
    "2022-11-02",
    "2023-02-02",
    "2023-05-02"
]

stock_init_price = {
    "GOOGL":  2461.48 / 20,
    "AAPL": 162.88,
    "MSFT": 280.72
}

stock_ratio = {
    "GOOGL":  40.62596486666558,
    "AAPL": 30.6974,
    "MSFT": 17.8113
}


In [77]:
## config
log = True

## Count Business days

In [59]:
def get_business_days_from_initial_date(cur_date, initial_date = issue_date):
    return np.busday_count(initial_date, cur_date)

def get_business_days_to_final_redemption(cur_date, final_date = final_redemption_date):
    return np.busday_count(cur_date, final_date)

In [60]:
print("coupon_payment_dates")
for date in coupon_payment_dates:
    print(date, ": ", get_business_days_from_initial_date(date))
    print("dates to marutre: ", get_business_days_to_final_redemption(date))

print("early_redemption_dates")
for date in early_redemption_dates:
    print(date, ": ", get_business_days_from_initial_date(date))
    print("dates to marutre: ", get_business_days_to_final_redemption(date))

coupon_payment_dates
2022-08-02 :  66
dates to marutre:  261
2022-11-02 :  132
dates to marutre:  195
2023-02-02 :  198
dates to marutre:  129
2023-05-02 :  261
dates to marutre:  66
2023-08-02 :  327
dates to marutre:  0
early_redemption_dates
2022-11-02 :  132
dates to marutre:  195
2023-02-02 :  198
dates to marutre:  129
2023-05-02 :  261
dates to marutre:  66


## Caculate interest rate discounted price

In [61]:
## dummy intesest rate func 
# (should return the average interest rate from current date to final redemption date)
def get_interest_rate(cur_date, to_date = final_redemption_date):
    return 0.03

def get_discounted_price(p: float, cur_date, to_date = final_redemption_date):
    r = get_interest_rate(cur_date)
    return p * np.exp(- r * np.busday_count(cur_date, to_date) / 250)

In [62]:
## simple test: how much is $100 one year later worth today?
get_discounted_price(100, "2022-11-07", "2023-11-07")

96.91653905244968

In [116]:
## p_mature should be the lowest total price of stocks
def get_product_price(p_mature, cur_date):
    coupon_remaining = 0.0
    for coupon_payment_date in coupon_payment_dates:
        if cur_date < coupon_payment_date:
            coupon_remaining += get_discounted_price(denomination * coupon_rate, cur_date, coupon_payment_date)
    d = get_discounted_price(p_mature, cur_date, final_redemption_date)
    return d + coupon_remaining


In [64]:
## How much is 5000 + 11% quarterly worth on issue_date?
get_product_price(5000, issue_date)

5661.0449427122585

In [65]:
5000 * (1 + 0.11 * 1.5)

5825.0

## Redemption Amount

In [66]:
df_init_price = pd.Series(stock_init_price)
df_convert = pd.Series(stock_ratio)
df_init_price, df_convert


(GOOGL    123.074
 AAPL     162.880
 MSFT     280.720
 dtype: float64,
 GOOGL    40.625965
 AAPL     30.697400
 MSFT     17.811300
 dtype: float64)

In [111]:
## input: 
stock_names = ["GOOGL", "AAPL", "MSFT"]

def get_redemption_amount(df: pd.DataFrame):
    lowest_prices = df[stock_names].min()
    barrier_trigger = (lowest_prices / df_init_price < barrier).sum() 

    if log:
        print("barrier_trigger", barrier_trigger)

    if barrier_trigger > 0:
        stock_min = (df.loc[df.index.max(), stock_names] * df_convert).min()
        if log:
            print("stock_min", stock_min)
        return min(stock_min, denomination)
    else:
        return denomination


In [112]:
sample_path_df = pd.read_csv("../generated_data/sample_path.csv", index_col = "date")
df = sample_path_df
df

,GOOGL,MSFT,AAPL
date,,,
2022-07-22,113.602065,262.053037,153.942500
2022-07-25,114.735530,264.850739,155.106186
2022-07-26,111.220134,254.515022,148.911027
2022-07-27,108.008860,247.503351,144.744386
2022-07-28,103.235604,235.304339,137.725854
...,...,...,...
2023-07-20,74.088786,182.356124,127.010467
2023-07-21,73.337796,179.753237,126.103559
2023-07-24,68.467933,168.802990,117.956346


In [113]:
get_redemption_amount(sample_path_df)

barrier_trigger 3
stock_min 2698.115894502515


2698.115894502515

In [117]:
get_product_price(get_redemption_amount(sample_path_df), cur_date = sample_path_df.index.min())

barrier_trigger 3
stock_min 2698.115894502515


3288.9911854453044

In [115]:
3363.8576424803377 / 5000

0.6727715284960676